### Using this code we will make transfer learning operation
### It's desirable to run code in Google Colaboratory
### Run cells consistently

In [0]:
#Importing all neccessary libraries
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model
import tensorflow as tf

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from google.colab import files

In [0]:
sess = None
graph=None
input_tensor = None
n_neighbours = None
predicted_labels = None
predicted_similarities = None
#There write output node names of your model
output_node_names= ["predicted_classes","predicted_nearest_idx"]

In [0]:
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

In [0]:
#There it is neccessary to indicate way to your saved graph in .h5 format on your Google Drive or computer (if you use Jupiter Notebook)
feature_extractor = load_model("/content/gdrive/My Drive/pdd_model_new/feature_extractor_all_150epochs.h5")

In [0]:
#Indicate number of classes for final recognition
N_CLASSES=30

In [0]:
base_model=feature_extractor
x=base_model.output
preds=Dense(N_CLASSES, activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

In [0]:
#It is optional
model.summary()

In [0]:
#Freezing your graph model last layer
for layer in model.layers[:-1]:
    layer.trainable=False

In [0]:
#Importing datasets for training model
!rm -r -f pdd_new
!git clone https://github.com/Kaliostrogoblin/PDD.git

In [0]:
import os
os.chdir('PDD')
# verify if we are in correct directory
os.listdir()

In [0]:
from pdd.datasets.allcrops import load_data
train_data, test_data = load_data(split_on_train_test=True, random_state=13)

In [0]:
from pdd.utils.data_utils import create_dataset_from_dir

train_dataset = create_dataset_from_dir(train_data, shuffle=True)
test_dataset = create_dataset_from_dir(test_data, shuffle=True)

In [0]:
#Checking that labels in both datasets are similar
print (test_dataset['target_names'])
print (train_dataset['target_names'])

In [0]:
#Importing libraries for starting training model
from keras.optimizers import SGD
!pip uninstall keras
!pip install git+git://github.com/keras-team/keras.git

In [0]:
#Parameters for final model
sgd = SGD(lr=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [0]:
from keras.utils import to_categorical
train_categorical = np.eye(N_CLASSES)[train_dataset['target']]
test_categorical  = np.eye(N_CLASSES)[test_dataset['target']]

In [0]:
#Training model
history=model.fit(x=train_dataset['data'],
          y=train_categorical, 
          batch_size=32,
          epochs=100,
          validation_data=(test_dataset['data'], test_categorical),
          shuffle=True)

In [0]:
#Saving a model
print("Saving feature extractor...")
#There you indicate path there model with transfer learning would be saved
model.save('/content/gdrive/My Drive/pdd_classifier_all_150epochs_100epochs.h5')